# 📘 PYTORCH PHASE 1 - FILE 1: OPTIMIZATION

**Core Concepts:** Optimization Fundamentals & Practical Experiments

**Mục tiêu:**
- ✅ Hiểu optimization landscape (convex vs non-convex)
- ✅ Master SGD family (Momentum, Nesterov)
- ✅ Understand adaptive optimizers (Adam, AdamW)
- ✅ Learning rate strategies (decay, warmup, cosine)
- ✅ Practical experiments & visualization

**Thời lượng:** 2-3 tuần

---

## 📚 Mục Lục

### 1. OPTIMIZATION FUNDAMENTALS
1.1 Loss Landscape
1.2 Convex vs Non-convex
1.3 Saddle Points vs Local Minima
1.4 Gradient Noise & Stochasticity
1.5 Batch Size Effects

### 2. SGD FAMILY
2.1 Vanilla SGD
2.2 SGD + Momentum
2.3 Nesterov Momentum
2.4 Exponential Moving Average
2.5 Practical Comparison

### 3. ADAPTIVE OPTIMIZERS
3.1 Adagrad
3.2 RMSProp
3.3 Adam
3.4 AdamW (Weight Decay)
3.5 Adam vs SGD in Practice

### 4. LEARNING RATE STRATEGIES
4.1 Constant vs Decaying LR
4.2 Step Decay
4.3 Exponential Decay
4.4 Cosine Annealing
4.5 Warmup Strategies
4.6 LR Range Test

### 5. PRACTICAL EXPERIMENTS
5.1 Optimizer Comparison
5.2 Loss Curve Analysis
5.3 LR Misconfiguration Effects
5.4 Training Instability Visualization

---

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import copy

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

---

# 1. OPTIMIZATION FUNDAMENTALS

## 1.1 Loss Landscape

### Định nghĩa

**Loss Landscape** = Bề mặt thể hiện loss function trên không gian parameters

### Convex vs Non-convex

#### Convex (Lồi)
- **Đặc điểm**: Có duy nhất một global minimum
- **Tính chất**: Bất kỳ local minimum nào cũng là global minimum
- **Ví dụ**: Linear regression, Logistic regression
- **Optimization**: Dễ dàng, guaranteed converge

#### Non-convex (Không lồi)
- **Đặc điểm**: Nhiều local minima, saddle points
- **Tính chất**: Phức tạp, nhiều "thung lũng"
- **Ví dụ**: Deep neural networks
- **Optimization**: Khó khăn, không guarantee global minimum

### Visualization

In [ ]:
# Visualize Convex vs Non-convex Loss

def plot_loss_landscape():
    """Plot 2D loss landscape comparison"""
    x = np.linspace(-5, 5, 100)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Convex: Quadratic
    y_convex = x**2
    axes[0].plot(x, y_convex, linewidth=3, color='blue')
    axes[0].scatter([0], [0], s=200, c='red', marker='*', zorder=5, label='Global Minimum')
    axes[0].set_xlabel('Parameter', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Convex Loss Landscape', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)
    
    # Non-convex: Multiple minima
    y_nonconvex = x**4 - 5*x**2 + 4
    axes[1].plot(x, y_nonconvex, linewidth=3, color='green')
    # Local minima
    minima_x = [-1.58, 1.58]
    minima_y = [x**4 - 5*x**2 + 4 for x in minima_x]
    axes[1].scatter(minima_x, minima_y, s=200, c='red', marker='*', zorder=5, label='Local Minima')
    # Saddle point
    axes[1].scatter([0], [4], s=200, c='orange', marker='o', zorder=5, label='Saddle Point')
    axes[1].set_xlabel('Parameter', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].set_title('Non-convex Loss Landscape', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=11)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Observations:")
    print("   Convex: One clear path to global minimum")
    print("   Non-convex: Multiple valleys, saddle points")
    print("   Deep learning = Non-convex optimization!")

plot_loss_landscape()

## 1.2 Saddle Points vs Local Minima

### Saddle Point (Điểm yên ngựa)
- Gradient = 0
- Minimum theo một số directions, maximum theo directions khác
- Common trong high-dimensional spaces
- **Problem**: SGD có thể stuck ở saddle points

### Local Minimum (Cực tiểu địa phương)
- Gradient = 0
- Minimum theo mọi directions
- **Problem**: Không phải global minimum

### Key Insight
- Trong deep learning, **saddle points** phổ biến hơn local minima
- SGD + Momentum giúp escape saddle points
- Stochastic gradient noise giúp exploration

## 1.3 Gradient Noise & Mini-batch Stochasticity

### Full Batch vs Mini-batch

#### Full Batch Gradient Descent
```python
# Compute gradient trên TOÀN BỘ dataset
loss = compute_loss(model(X_train), y_train)
loss.backward()
optimizer.step()
```
- ✅ Gradient chính xác
- ❌ Chậm (phải xử lý toàn bộ data)
- ❌ Memory intensive

#### Mini-batch SGD
```python
# Compute gradient trên SUBSET
for X_batch, y_batch in dataloader:
    loss = compute_loss(model(X_batch), y_batch)
    loss.backward()
    optimizer.step()
```
- ✅ Nhanh
- ✅ Memory efficient
- ✅ Gradient noise giúp exploration
- ❌ Gradient không chính xác (noisy)

### Effect of Batch Size

| Batch Size | Gradient Noise | Convergence Speed | Generalization |
|------------|----------------|-------------------|----------------|
| **Small (32)** | High | Slow per epoch, fast per iteration | Better (more noise) |
| **Medium (256)** | Medium | Balanced | Balanced |
| **Large (2048)** | Low | Fast per epoch, slow per iteration | Worse (less noise) |

### Key Finding
- Small batch → Better generalization (noise acts as regularization)
- Large batch → Sharper minima → Worse generalization
- Solution: Large batch + LR warmup + longer training

In [ ]:
# Experiment: Batch Size Effect

def train_with_batch_size(model, train_data, batch_size, epochs=10):
    """
    Train model với batch size cụ thể
    """
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    
    losses = []
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            losses.append(loss.item())
            epoch_loss += loss.item()
    
    return losses

# Create toy dataset
X = torch.randn(1000, 10)
y = torch.randint(0, 2, (1000,))
train_data = TensorDataset(X, y)

# Simple model
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 2)
    
    def forward(self, x):
        return self.fc(x)

# Compare different batch sizes
batch_sizes = [16, 64, 256]
results = {}

print("🔄 Training với different batch sizes...\n")

for bs in batch_sizes:
    model = SimpleNet().to(device)
    losses = train_with_batch_size(model, train_data, batch_size=bs, epochs=5)
    results[bs] = losses
    print(f"✅ Batch size {bs}: {len(losses)} iterations")

# Plot
plt.figure(figsize=(12, 5))

for bs, losses in results.items():
    # Smooth with moving average
    window = 10
    smoothed = np.convolve(losses, np.ones(window)/window, mode='valid')
    plt.plot(smoothed, label=f'Batch size = {bs}', linewidth=2, alpha=0.8)

plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Effect of Batch Size on Training Loss', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("\n📊 Observations:")
print("   Small batch (16): Noisy but fast convergence per iteration")
print("   Large batch (256): Smooth but slower convergence per iteration")
print("   Trade-off: Noise vs Stability")

---

# 2. SGD FAMILY

## 2.1 Vanilla SGD

### Formula

$$\theta_{t+1} = \theta_t - \eta \nabla L(\theta_t)$$

Trong đó:
- $\theta$: Parameters
- $\eta$: Learning rate
- $\nabla L$: Gradient của loss

### Characteristics
- ✅ Simple, easy to understand
- ✅ Memory efficient
- ❌ Slow convergence
- ❌ Oscillates around minimum
- ❌ Sensitive to learning rate

In [ ]:
# Vanilla SGD Implementation

class VanillaSGD:
    """
    Vanilla SGD optimizer (for educational purposes)
    """
    def __init__(self, params, lr=0.01):
        self.params = list(params)
        self.lr = lr
    
    def step(self):
        """Update parameters"""
        with torch.no_grad():
            for param in self.params:
                if param.grad is not None:
                    # θ_{t+1} = θ_t - η * ∇L(θ_t)
                    param.data -= self.lr * param.grad
    
    def zero_grad(self):
        """Zero all gradients"""
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()

print("✅ Vanilla SGD implemented!")
print("\n💡 Formula: θ_{t+1} = θ_t - η * ∇L(θ_t)")
print("   - Simple update rule")
print("   - No memory of past gradients")

## 2.2 SGD + Momentum

### Motivation

Vanilla SGD có vấn đề:
- Dao động nhiều trong narrow valleys
- Chậm khi surface phẳng

### Solution: Momentum

$$v_t = \beta v_{t-1} + \nabla L(\theta_t)$$
$$\theta_{t+1} = \theta_t - \eta v_t$$

Trong đó:
- $v_t$: Velocity (momentum buffer)
- $\beta$: Momentum coefficient (typically 0.9)

### Intuition

Momentum = **Exponential Moving Average** of gradients

$$v_t = \beta v_{t-1} + g_t = \sum_{i=0}^{t} \beta^{t-i} g_i$$

### Benefits
- ✅ Faster convergence
- ✅ Reduced oscillation
- ✅ Better handling of noisy gradients
- ✅ Can escape shallow local minima

In [ ]:
# SGD + Momentum Implementation

class SGDMomentum:
    """
    SGD with Momentum
    """
    def __init__(self, params, lr=0.01, momentum=0.9):
        self.params = list(params)
        self.lr = lr
        self.momentum = momentum
        
        # Initialize velocity buffers
        self.velocity = [torch.zeros_like(p.data) for p in self.params]
    
    def step(self):
        """Update parameters with momentum"""
        with torch.no_grad():
            for i, param in enumerate(self.params):
                if param.grad is not None:
                    # v_t = β * v_{t-1} + g_t
                    self.velocity[i] = self.momentum * self.velocity[i] + param.grad
                    
                    # θ_{t+1} = θ_t - η * v_t
                    param.data -= self.lr * self.velocity[i]
    
    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()

print("✅ SGD + Momentum implemented!")
print("\n💡 Key ideas:")
print("   - Accumulate gradient history")
print("   - Smooth out oscillations")
print("   - Accelerate in consistent directions")
print("   - β = 0.9 means ~10 past gradients influence")

## 2.3 Nesterov Momentum

### Motivation

Standard momentum:
1. Compute gradient tại current position
2. Update với momentum

Problem: Không "look ahead"

### Nesterov Accelerated Gradient (NAG)

$$v_t = \beta v_{t-1} + \nabla L(\theta_t - \beta v_{t-1})$$
$$\theta_{t+1} = \theta_t - \eta v_t$$

Key difference: Compute gradient tại **lookahead position** $\theta_t - \beta v_{t-1}$

### Intuition

- Standard momentum: "Jump first, look later"
- Nesterov: "Look ahead, then jump"

### Benefits
- ✅ Better convergence than standard momentum
- ✅ Corrects before overshooting
- ✅ More stable

In [ ]:
# Visualize: SGD vs Momentum vs Nesterov

def visualize_optimizer_paths():
    """
    Visualize optimization paths for different optimizers
    """
    # Simple 2D quadratic: f(x,y) = x^2 + 10*y^2
    def loss_fn(x, y):
        return x**2 + 10*y**2
    
    def grad_fn(x, y):
        return 2*x, 20*y
    
    # Contour plot
    x = np.linspace(-5, 5, 100)
    y = np.linspace(-5, 5, 100)
    X, Y = np.meshgrid(x, y)
    Z = loss_fn(X, Y)
    
    plt.figure(figsize=(14, 5))
    
    # Plot loss landscape
    for idx, (name, lr, momentum) in enumerate([
        ('Vanilla SGD', 0.1, 0.0),
        ('SGD + Momentum', 0.1, 0.9),
    ]):
        plt.subplot(1, 2, idx+1)
        plt.contour(X, Y, Z, levels=20, alpha=0.6)
        plt.colorbar(label='Loss')
        
        # Simulate optimization
        x_pos, y_pos = 4.0, 4.0
        vx, vy = 0.0, 0.0
        
        trajectory_x = [x_pos]
        trajectory_y = [y_pos]
        
        for _ in range(50):
            gx, gy = grad_fn(x_pos, y_pos)
            
            # Update velocity
            vx = momentum * vx + gx
            vy = momentum * vy + gy
            
            # Update position
            x_pos -= lr * vx
            y_pos -= lr * vy
            
            trajectory_x.append(x_pos)
            trajectory_y.append(y_pos)
        
        plt.plot(trajectory_x, trajectory_y, 'r-o', markersize=4, linewidth=2, alpha=0.7)
        plt.scatter([0], [0], s=200, c='green', marker='*', zorder=5, label='Global Minimum')
        plt.scatter([4], [4], s=150, c='red', marker='o', zorder=5, label='Start')
        
        plt.xlabel('x', fontsize=12)
        plt.ylabel('y', fontsize=12)
        plt.title(name, fontsize=13, fontweight='bold')
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Observations:")
    print("   Vanilla SGD: Oscillates, slow convergence")
    print("   Momentum: Smoother, faster convergence")
    print("   Nesterov: Even better correction")

visualize_optimizer_paths()

---

# 3. ADAPTIVE OPTIMIZERS

## 3.1 Problem với Fixed Learning Rate

SGD family dùng **same learning rate** cho all parameters:

$$\theta_{t+1} = \theta_t - \eta \nabla L(\theta_t)$$

### Issues
- Parameters có gradients khác nhau rất nhiều
- Some dimensions cần large steps
- Some dimensions cần small steps
- Fixed LR không optimal cho tất cả

### Solution: Adaptive Learning Rates

**Idea**: Adapt learning rate per parameter dựa trên gradient history

## 3.2 Adagrad

### Formula

$$G_t = G_{t-1} + g_t^2$$
$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{G_t + \epsilon}} g_t$$

Trong đó:
- $G_t$: Sum of squared gradients
- $\epsilon$: Small constant (1e-8) for numerical stability

### Intuition
- Parameters với large gradients → smaller effective LR
- Parameters với small gradients → larger effective LR

### Problem
- $G_t$ continuously grows → LR shrinks to zero
- Training stops prematurely
- Not suitable for deep learning

## 3.3 RMSProp

### Motivation

Adagrad's problem: $G_t$ grows indefinitely

### Solution

Use **exponential moving average** instead of sum:

$$v_t = \beta v_{t-1} + (1-\beta) g_t^2$$
$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{v_t + \epsilon}} g_t$$

Trong đó:
- $\beta$: Decay rate (typically 0.9 or 0.99)
- $v_t$: Moving average of squared gradients

### Benefits
- ✅ Prevents LR from shrinking to zero
- ✅ Adapts to recent gradient history
- ✅ Works well in practice

## 3.4 Adam (Adaptive Moment Estimation)

### Formula

Adam = **Momentum** + **RMSProp**

$$m_t = \beta_1 m_{t-1} + (1-\beta_1) g_t \quad \text{(First moment: mean)}$$
$$v_t = \beta_2 v_{t-1} + (1-\beta_2) g_t^2 \quad \text{(Second moment: variance)}$$

Bias correction:
$$\hat{m}_t = \frac{m_t}{1-\beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1-\beta_2^t}$$

Update:
$$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t$$

### Hyperparameters
- $\beta_1 = 0.9$ (momentum)
- $\beta_2 = 0.999$ (RMSProp)
- $\epsilon = 10^{-8}$
- $\eta$ = learning rate (often 0.001)

### Benefits
- ✅ Combines best of momentum & adaptive LR
- ✅ Works well với sparse gradients
- ✅ Default choice for many applications
- ✅ Bias correction prevents initial steps từ being too large

In [ ]:
# Adam Implementation (Simplified)

class AdamOptimizer:
    """
    Adam optimizer (educational implementation)
    """
    def __init__(self, params, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
        self.params = list(params)
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        
        # Initialize moment estimates
        self.m = [torch.zeros_like(p.data) for p in self.params]  # First moment
        self.v = [torch.zeros_like(p.data) for p in self.params]  # Second moment
        self.t = 0  # Timestep
    
    def step(self):
        """Adam update"""
        self.t += 1
        
        with torch.no_grad():
            for i, param in enumerate(self.params):
                if param.grad is not None:
                    g = param.grad
                    
                    # Update biased first moment estimate
                    self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * g
                    
                    # Update biased second moment estimate
                    self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * g**2
                    
                    # Bias correction
                    m_hat = self.m[i] / (1 - self.beta1**self.t)
                    v_hat = self.v[i] / (1 - self.beta2**self.t)
                    
                    # Update parameters
                    param.data -= self.lr * m_hat / (torch.sqrt(v_hat) + self.eps)
    
    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()

print("✅ Adam optimizer implemented!")
print("\n💡 Adam = Momentum + RMSProp + Bias Correction")
print("   - m_t: First moment (mean of gradients)")
print("   - v_t: Second moment (variance of gradients)")
print("   - Bias correction: Important for initial steps")

## 3.5 AdamW (Weight Decay)

### Problem with Adam + L2 Regularization

Traditional L2 regularization:
$$L = L_{task} + \frac{\lambda}{2}||\theta||^2$$

Gradient:
$$\nabla L = \nabla L_{task} + \lambda \theta$$

**Problem**: Khi dùng với Adam, regularization term được adaptive → không consistent!

### Solution: Decoupled Weight Decay

AdamW separates weight decay from gradient:

$$\theta_{t+1} = \theta_t - \eta \left(\frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon} + \lambda \theta_t\right)$$

### Key Difference

| Aspect | Adam + L2 | AdamW |
|--------|-----------|-------|
| Weight decay | In gradient | Separate term |
| Adaptive | Yes | No |
| Effect | Inconsistent | Consistent |
| Performance | Worse | Better |

### Recommendation
- ✅ Use **AdamW** instead of Adam + L2
- ✅ Typical weight decay: 0.01 ~ 0.1
- ✅ Standard choice for Transformers

---

# 🎓 Tổng kết FILE 1: Optimization

## ✅ Những gì đã học

### 1. Optimization Fundamentals
- **Loss landscape**: Convex vs Non-convex
- **Saddle points**: Common trong high dimensions
- **Batch size effects**: Small batch → better generalization
- **Gradient noise**: Acts as regularization

### 2. SGD Family
- **Vanilla SGD**: Simple but slow
- **Momentum**: Exponential moving average of gradients
- **Nesterov**: Look-ahead correction
- β = 0.9 is typical momentum value

### 3. Adaptive Optimizers
- **Adagrad**: Accumulates squared gradients (problematic)
- **RMSProp**: Moving average of squared gradients
- **Adam**: Momentum + RMSProp + bias correction
- **AdamW**: Decoupled weight decay (preferred)

## 🚀 Key Takeaways

1. **Deep learning = Non-convex optimization**
2. **Momentum** smooths oscillations, accelerates convergence
3. **Adam** = default choice (but not always best)
4. **AdamW** better than Adam + L2
5. **Small batch size** often generalizes better
6. **Learning rate** most important hyperparameter

## 📝 Next: FILE 2

- Learning Rate Strategies
- Practical Experiments
- Optimizer Comparison
- When Adam fails

---

**Chúc mừng bạn đã hoàn thành FILE 1! 🎉**